<a href="https://colab.research.google.com/github/bhashitha69/hellow_test/blob/main/Dact_me_contr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from numpy.linalg import matrix_power
from numpy.linalg import inv
from scipy.linalg import null_space
from sympy import Matrix

def c2dm(A_c,B_c,C_c,D_c,Delta_t):
  A_d = (Delta_t*A_c) + np.eye(A_c.shape[0]);
  B_d = Delta_t*B_c;
  C_d = C_c;
  D_d = D_c;
  return [A_d, B_d, C_d, D_d]

def augment(Am,Bm,Cm):
  Om = np.zeros((Am.shape[0],Am.shape[1]));
  A_c1 = np.concatenate((Am,Om.transpose()),axis = 1)
  A_c2 = np.concatenate((Cm@Am,np.eye(Am.shape[0])), axis = 1)
  A = np.concatenate((A_c1,A_c2),axis = 0)
  B = np.concatenate((Bm,Cm@Bm), axis = 0)
  C = np.concatenate((Om,np.eye(Am.shape[0])),axis = 1)
  return [A, B, C]

def hessff(A,B,C,n_p):
  hes1 = np.zeros((C.shape[0],B.shape[1]))
  ff = np.empty([0,A.shape[1]])
  hes = np.empty([C.shape[0],0])
  hessi = np.empty([0,B.shape[1]*n_p])
  for i in range(0,n_p):
    hes = np.empty([C.shape[0],0])
    for n in range(0,n_p):
      if n <= i:
        hess = C@matrix_power(A,i-n)@B
      else:
        hess = hes1
      hes = np.concatenate((hes,hess),axis = 1)
    hessi = np.concatenate((hessi,hes),axis = 0)
    ff = np.concatenate((ff,C@matrix_power(A,i+1)),axis = 0)
  return [ff , hessi]

def bar_R(r, N):
  r_0 = np.full((N,len(r)),r)
  r_1 = np.reshape(r_0,N*len(r))
  R = np.diag(r_1)
  return R

def out_Rs(ref,N):
  r_s = np.full((N, len(ref)),ref)
  rs1 = np.reshape(r_s, (N*len(ref) , 1))
  return rs1

def get_C_12(B,N):
  n = B.shape[1]
  i_C = np.eye(n)
  iop =np.full((N, n, n),i_C)
  hes1 = np.zeros((n,n))
  hessi = np.empty([0,n*N])
  ff = np.reshape(iop, (N*n , n))
  for i in range(0,N):
    hes = np.empty([n,0])
    for j in range(0,N):
      if j <= i:
        hess = i_C
      else:
        hess = hes1

      hes = np.concatenate((hes,hess),axis = 1)
    hessi = np.concatenate((hessi,hes),axis = 0)
  return [ff, hessi]

In [ ]:
def get_lmd(E,M_c,F,f_f):
  K_1 = np.concatenate((E,M_c.T), axis = 1)
  K_1.shape[1]
  K_2 = np.concatenate((M_c,np.zeros((M_c.shape[0],M_c.T.shape[1]))), axis = 1)
  K_k = np.concatenate((K_1,K_2), axis = 0)
  F_f = np.concatenate((-F,f_f), axis = 0)
  K_k_1 = np.array(inv(K_k),dtype = "float32")
  L_l = K_k_1@F_f
  return np.array(L_l[L_l.shape[0] - M_c.shape[0]:], dtype = "float32")

def get_Xs(E,M_c,F,f_f):
  K_1 = np.concatenate((E,M_c.T), axis = 1)
  K_1.shape[1]
  K_2 = np.concatenate((M_c,np.zeros((M_c.shape[0],M_c.T.shape[1]))), axis = 1)
  K_k = np.concatenate((K_1,K_2), axis = 0)
  F_f = np.concatenate((-F,f_f), axis = 0)
  K_k_1 = np.array(inv(K_k),dtype = "float32")
  L_l = K_k_1@F_f
  return np.array(L_l[0:M_c.shape[1]], dtype = "float32")

In [ ]:
def dfunc(X):
  m = 1; g = 1; l = 1; I = 1; k = 1; j = 1
  df = np.array([[0, 1, 0, 0],[-m*g*l*np.cos(X[0][0])/I - k, 0, k, 0],[0, 0, 0, 1],[k/j, 0, -k/j, 0]])
  return df

In [ ]:
a = np.array([[0, 1, 0],[3, 0, 1], [0, 1, 0]])
b = np.array([[1, 1, 13],[1, -4, 2], [3, 1, -5]])
c = np.array([[1, 0, 0],[0, 1, 0],[0, 0, 1]])
d = np.array([[0, 0, 0]])
Q_q = np.array([[1, 0, 0],[0, 1, 0], [0, 0, 1]])
Q_q = bar_R([0.0001, 1, 0.001],20)
t = 0.02
N = 20
[Am,Bm,Cm,Dm] = c2dm(a,b,c,d,t)
[A, B, C] = augment(Am,Bm,Cm)
[f, he] = hessff(A,B,C,N)

Rr = bar_R([0.07, 0.07, 0.07],N)
R_s = out_Rs([1, -1, 1],N)

x = np.array([[1],[-3],[1]])
x_1 = Am@x
x_2 = x_1 - x
x = np.concatenate((x_2.T,x_1.T),axis = 1)
jkl = (inv(he.T@Q_q@he + Rr))@he.T@Q_q@(R_s - (f@(x.T)))

In [ ]:
iopp = jkl.reshape(N,3)
i1op = np.zeros((N,3))
for i1 in range(0,N):
  i1op[i1][0] = iopp[0:i1,0].sum()
  i1op[i1][1] = iopp[0:i1,1].sum()
  i1op[i1][2] = iopp[0:i1,2].sum()
np.round(i1op,decimals = 2)

array([[ 0.  ,  0.  ,  0.  ],
       [ 0.91, -3.89,  1.49],
       [ 1.01, -4.64,  1.28],
       [ 0.76, -3.88,  0.47],
       [ 0.43, -2.67, -0.36],
       [ 0.16, -1.56, -0.95],
       [-0.01, -0.77, -1.24],
       [-0.08, -0.32, -1.29],
       [-0.08, -0.13, -1.19],
       [-0.04, -0.12, -1.  ],
       [ 0.01, -0.21, -0.8 ],
       [ 0.07, -0.32, -0.62],
       [ 0.12, -0.43, -0.46],
       [ 0.16, -0.53, -0.34],
       [ 0.19, -0.61, -0.24],
       [ 0.21, -0.66, -0.18],
       [ 0.23, -0.7 , -0.13],
       [ 0.24, -0.73, -0.1 ],
       [ 0.25, -0.76, -0.08],
       [ 0.25, -0.77, -0.07]])

In [ ]:
Iopp = jkl.T[0]
Iiop = np.zeros((N+1,3))
Xm = np.array([[1],[-3],[1]])
for i1 in range(0,N):
  X_v = Cm@Xm
  Iiop[i1][0] = X_v[0][0]
  Iiop[i1][1] = X_v[1][0]
  Iiop[i1][2] = X_v[2][0]

  Xmn = Am@Xm + Bm@i1op[i1].reshape(3,1)
  #print(Bmi1op[i1].reshape(3,1))
  Xm = Xmn
np.round(Iiop,decimals = 2)

array([[ 1.  , -3.  ,  1.  ],
       [ 0.94, -2.92,  0.94],
       [ 1.21, -2.46,  0.71],
       [ 1.42, -1.93,  0.5 ],
       [ 1.44, -1.49,  0.38],
       [ 1.27, -1.19,  0.36],
       [ 0.97, -1.01,  0.41],
       [ 0.62, -0.93,  0.5 ],
       [ 0.25, -0.91,  0.6 ],
       [-0.08, -0.93,  0.69],
       [-0.36, -0.95,  0.77],
       [-0.59, -0.97,  0.83],
       [-0.78, -0.99,  0.87],
       [-0.92, -1.  ,  0.89],
       [-1.04, -1.  ,  0.9 ],
       [-1.13, -1.  ,  0.91],
       [-1.2 , -1.  ,  0.9 ],
       [-1.27, -1.  ,  0.9 ],
       [-1.32, -1.  ,  0.89],
       [-1.38, -1.  ,  0.87],
       [ 0.  ,  0.  ,  0.  ]])

In [ ]:
E = he.T@Q_q@he + Rr
R_T = np.linalg.cholesky(E)
R = R_T.T

In [ ]:
def t_maxx(A,B):

  R = np.zeros((A.shape[0],1))
  for i in range(0,A.shape[0]):
    if abs(A[i][0]) >= 0.000001:
      r = B[i][0]/A[i][0]
      if r <= 1 and r >= 0:
        R[i][0] = r

  return np.amax(R) , np.argmax(R)

In [ ]:
def get_minind(X123):
  J1 = []
  for i3 in range(0,X123.shape[0]):
    if X123[i3][0] < 0:
      J1.append(i3)
  return J1

In [ ]:
def fixComp(lmd,B,pp_k,WW,BBb):
  #print(lmd)
  #print(pp_k[B])
  r = np.argmin(-lmd[B]/pp_k[B])
  #print(r)
  min_p = -lmd[B[r]]/pp_k[B[r]]
  #print(min_p)
  min = min_p[0]

  LmD = lmd + min*pp_k
  WW.remove(B[r])
  BBb.append(B[r])
  return LmD, WW, BBb

In [ ]:
Y_min = np.full((N,3,1),[[0.5],[-4],[0.5]])
Y_min = Y_min.reshape(N*3,1)
Y_max = np.full((N,3,1),[[1.5],[0],[1.5]])
Y_max = Y_max.reshape(N*3,1)

In [ ]:
u_k_1 = np.array([[0],[0],[0]])
#u = np.zeros((1,100))
#U = np.zeros((1,100))
x_t = x.T
#X = np.empty([4,0])
[C_1 , C_2] = get_C_12(B,N)
II = np.eye(3).tolist()
ll = np.full((N,3,3),[[1, 0, 0],[0, 1, 0],[0, 0, 1]])
ll = ll.reshape(N*3,3)
Umin = ll@np.array([[1],[3],[0.5]]) + C_1@u_k_1
Umax = ll@np.array([[1],[3],[0.5]]) - C_1@u_k_1
M = np.concatenate((-C_2,C_2), axis = 0)
M_1 = np.concatenate((-he,he), axis = 0)
Y_min = -Y_min + f@x_t
Y_max = Y_max - f@x_t
Y_lim = np.concatenate((Y_min,Y_max), axis = 0)
M = np.concatenate((M,M_1),axis = 0)
M = np.array(M, dtype = "float16")
gamm = np.concatenate((Umin,Umax), axis = 0)
gamm = np.concatenate((gamm,Y_lim), axis = 0)
gamm = np.array(gamm, dtype = "float16")
E = he.T@Q_q@he + Rr
E_1 = inv(E)

F = -he.T@Q_q@(R_s - (f@(x_t)))

lmd_p = np.zeros((N*4*3,1), dtype = "float16")

Wq = [0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89 ,120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179]
bW = [30,31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49,  20, 21, 22, 23, 24, 25, 26, 27, 28, 29,  90,  91,  92,  93,  94,  95,  96,  97,  98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119,180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239]
#h = np.linalg.det(M@E_1@M.T)

#print(h)
#Lmd = -inv(M@E_1@M.T)@(gamm + M@E_1@F)
#dU = (inv(he.T@he + Rr))@he.T@(R_s - (f@(x_t)))
#U_0 = dU - E_1@M.T@Lmd
#x_n = A@(x_t) + B*dU[0][0]
#X = np.concatenate((X,x_n),axis = 1)
#x_t = x_n

In [ ]:
def D_act_set(A, b, P , Q, R_i,R, Lmd_0,w_0,bw_0):
  LMD = Lmd_0
  k = 0
  w = w_0
  bw = bw_0
  while True:
    print(bw)
    #print(lmd_k)
    A_k = A[w]
    M_k = A_k@R_i
    if (np.linalg.det(M_k@M_k.T) != 0.0):
      f = Q
      d_k = b[w] + M_k@inv(R.T)@f
      lmd_k = np.zeros((A.shape[0],1))
      print(lmd_k)
      lmd_k[w] = -inv(M_k@M_k.T)@d_k
      if (lmd_k >= 0.000).all() == True:
        Ub_u_k = np.zeros((A.shape[0],1))
        print(M_k)
        Ub_M_k = A[bw]@R_i
        Ub_d_k = b[bw] + Ub_M_k@inv(R.T)@f
        Ub_u_k[bw] = Ub_M_k@M_k.T@lmd_k[w] + Ub_d_k
        LMD = lmd_k
        k = k + 1

        if (Ub_u_k >= 0.000).all() :
          print(Ub_u_k)
          x_opt = -R_i@(M_k.T@lmd_k[w] + inv(R.T)@f)
          return x_opt, lmd_k
        else:
          U_n = np.argmin(Ub_u_k)
          w.append(U_n)
          bw.remove(U_n)
      else:
        p_k = lmd_k - LMD
        #B_0 = np.flatnonzero(lmd_k)
        #print(lmd_k)
        #B = B_0.tolist()
        B = get_minind(lmd_k)
        #print(lmd_k)
        LMD , w, bw = fixComp(LMD,B,p_k,w,bw)
        k = k + 1
    else:
      n_s = null_space(M_k@M_k.T)
      d = b[w] + A[w]@R_i@inv(R.T)@Q
      pre_dp = d.T@n_s
      pre_dp1 = pre_dp[0,1:]@np.ones((pre_dp.shape[1] - 1,1))
      #print(n_s)
      x_1 = (-0.5 - pre_dp1[0])/pre_dp[0][0]
      p_o_p = np.concatenate((np.array([[x_1]]),np.ones((pre_dp.shape[1] - 1,1))), axis = 0)
      p_s = n_s@p_o_p
      ppp_k = np.zeros((A.shape[0],1))
      ppp_k[w] = p_s
      #B_0 = np.flatnonzero(ppp_k)
      #B = B_0.tolist()
      #print(p_s)

      B = get_minind(ppp_k)
      LMD , w, bw = fixComp(LMD,B,ppp_k,w,bw)
      k = k + 1

In [ ]:
x_o, lmd_o = D_act_set(M,gamm,E,F,inv(R),R,lmd_p,Wq,bW)

In [ ]:
x_o.shape

(60, 1)

In [ ]:
x_o.reshape(20,3)

array([[ 1.00000000e+00, -3.00000000e+00,  5.00000000e-01],
       [ 2.68673972e-14,  7.66331443e-14,  5.88418203e-14],
       [ 6.56141808e-14,  9.90874049e-14,  4.52970994e-14],
       [-4.23091630e-02,  8.00712430e-02, -7.45456051e-02],
       [-2.05061023e-01,  7.96737035e-01, -3.52040162e-01],
       [-2.18498513e-01,  9.35535095e-01, -3.17966007e-01],
       [-1.66444789e-01,  7.92831112e-01, -1.71306316e-01],
       [-9.97496023e-02,  5.60178756e-01, -2.28355178e-02],
       [-4.35468464e-02,  3.40325022e-01,  7.91893186e-02],
       [-6.10401038e-03,  1.74212679e-01,  1.24570435e-01],
       [ 1.34051620e-02,  6.68570418e-02,  1.21855653e-01],
       [ 1.94709807e-02,  6.98005569e-03,  8.53901017e-02],
       [ 1.70451111e-02, -2.07565715e-02,  2.88123846e-02],
       [ 1.42115227e-02, -2.52119636e-02,  1.62790833e-02],
       [ 1.04022211e-02, -1.97853229e-02,  1.05444784e-02],
       [ 6.52873923e-03, -1.18199756e-02,  4.95704651e-03],
       [ 3.43021485e-03, -4.88233483e-03

In [ ]:
iopp = x_o.reshape(N,3)
i1op = np.zeros((N,3))
for i1 in range(0,N):
  i1op[i1][0] = iopp[0:i1,0].sum()
  i1op[i1][1] = iopp[0:i1,1].sum()
  i1op[i1][2] = iopp[0:i1,2].sum()
np.round(i1op,decimals = 2)

array([[ 0.  ,  0.  ,  0.  ],
       [ 1.  , -3.  ,  0.5 ],
       [ 1.  , -3.  ,  0.5 ],
       [ 1.  , -3.  ,  0.5 ],
       [ 0.96, -2.92,  0.43],
       [ 0.75, -2.12,  0.07],
       [ 0.53, -1.19, -0.24],
       [ 0.37, -0.39, -0.42],
       [ 0.27,  0.17, -0.44],
       [ 0.22,  0.51, -0.36],
       [ 0.22,  0.68, -0.23],
       [ 0.23,  0.75, -0.11],
       [ 0.25,  0.75, -0.03],
       [ 0.27,  0.73,  0.  ],
       [ 0.28,  0.71,  0.02],
       [ 0.29,  0.69,  0.03],
       [ 0.3 ,  0.68,  0.03],
       [ 0.3 ,  0.67,  0.03],
       [ 0.3 ,  0.67,  0.03],
       [ 0.3 ,  0.67,  0.03]])

In [ ]:
Iopp = jkl.T[0]
Iiop = np.zeros((N+1,3))
Xm = np.array([[1],[-3],[1]])
for i1 in range(0,N):
  X_v = Cm@Xm
  Iiop[i1][0] = X_v[0][0]
  Iiop[i1][1] = X_v[1][0]
  Iiop[i1][2] = X_v[2][0]

  Xmn = Am@Xm + Bm@i1op[i1].reshape(3,1)
  #print(Bmi1op[i1].reshape(3,1))
  Xm = Xmn
np.round(Iiop,decimals = 2)

array([[ 1.  , -3.  ,  1.  ],
       [ 0.94, -2.92,  0.94],
       [ 0.97, -2.56,  0.83],
       [ 1.01, -2.21,  0.73],
       [ 1.06, -1.85,  0.64],
       [ 1.09, -1.51,  0.56],
       [ 1.05, -1.24,  0.52],
       [ 0.95, -1.07,  0.53],
       [ 0.82, -0.99,  0.56],
       [ 0.7 , -0.95,  0.61],
       [ 0.6 , -0.95,  0.65],
       [ 0.54, -0.96,  0.68],
       [ 0.51, -0.97,  0.7 ],
       [ 0.5 , -0.98,  0.71],
       [ 0.5 , -0.99,  0.72],
       [ 0.5 , -1.  ,  0.73],
       [ 0.51, -1.  ,  0.74],
       [ 0.52, -1.  ,  0.75],
       [ 0.53, -1.  ,  0.76],
       [ 0.54, -1.  ,  0.77],
       [ 0.  ,  0.  ,  0.  ]])

In [ ]:
iopp = x_o
iiop = np.zeros((1,11))
for i1 in range(1,11):
  iiop[0][i1] = iopp[0:i1].sum()
iiop.T

array([[  0.        ],
       [ 39.55755803],
       [ 69.47994876],
       [ 90.99681351],
       [105.32473067],
       [113.63086059],
       [117.00669151],
       [116.45037121],
       [112.85618632],
       [107.00985739],
       [ 99.58844137]])

In [ ]:
X_x = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

In [ ]:
X_x = X_x.reshape(1,11)
Iiop = Iiop.reshape(1,11)
IIop = IIop.reshape(1,11)

In [ ]:
Iiop

array([[0.        , 0.        , 0.99335019, 2.35128288, 3.60802981,
        4.53246345, 5.08358766, 5.3276373 , 5.36191002, 5.26445466,
        5.07254392]])

In [ ]:
IIop

array([[0.        , 0.        , 0.8       , 2.08      , 3.35472466,
        4.3466438 , 4.97594005, 5.28547769, 5.36318274, 5.28645546,
        5.09518517]])

In [ ]:
fig2 = go.Figure()
fig2.add_trace(go.Scatter(x= X_x[0], y= Iiop[0][1:], mode='lines',name='$\omega_{J}\\text{ rad/s}$', marker=dict(color="blue")))
fig2.add_trace(go.Scatter(x= X_x[0], y= IIop[0][1:], mode='lines', name='$\omega_{J}\\text{ rad/s}$', marker=dict(color="red")))
fig2.update_layout(title='$\\text{ Input Voltages to Motors} $',xaxis=dict(title="$\\text{Time\\ } s$"),yaxis=dict(title="$V$"))

In [ ]:
fig = plt.figure(figsize =(10, 7))

# Horizontal Bar Plot
plt.bar(name[0:10], price[0:10])

# Show Plot
plt.show()

In [ ]:
databases = ['MySQL', 'MongoDB', 'Elasticsearch', 'Redis', 'SQLite']
usage = [80, 75, 42, 78, 60]
fig = go.Figure(data=go.Bar(
    x=X_x[0],
    y=i1op[0][1:]
))
fig.update_layout( bargap=0.0)
fig.show()

In [ ]:

databases = ['MySQL', 'MongoDB', 'Elasticsearch', 'Redis', 'SQLite']
usage = [80, 75, 42, 78, 60]
fig = go.Figure(data=go.Bar(
    x=X_x[0],
    y=iiop[0][1:]
))
fig.update_layout( bargap=0.0)
fig.show()

In [ ]:
iiop[0][1:]

array([ 8.        ,  8.        ,  2.82724664, -2.52915661, -5.97978137,
       -7.32815619, -7.23996444, -6.55450003, -5.91250329, -5.61328191])

In [ ]:
X_x

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10]])